# Joint Training Hybrid Model
The second approach is to have the regression model to incorporate in its training and testing the result of the classification model. The objective is to incorporate the ground truth of of the classification data set into the training data of the regression data set and demonstrated, that is, allow the model to learn to predict the amplitude of a constraint, given that that constraint exists. 

During testing, the truthful constraint violation Boolean feature is substituted by the prediction of the classification model, and the 
model will predict the amplitude of the constraint violation, given the prediction of its occurrence. 

## Model selection
Considering the datasets benchmark, the best regression model for max_u and min_u are: 
- max_u: lr focused
- min_u: gb sparse

The best classification model for max_u and min_u are:
- max_u: xgb balanced
- min_u: gb sparse


In [15]:
import sys;sys.path.append('..');from thesis_package import aimodels as my_ai, utils, metrics
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import os

## Load data

In [16]:
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])

In [17]:
# Classification data balanced
y_max_u_balanced_class = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_bool_constr.csv')
exogenous_data_balanced_max_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
y_min_u_balanced_class = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_bool_constr.csv')
exogenous_data_balanced_min_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
y_max_u_balanced_class = y_max_u_balanced_class[utils.cols_with_positive_values(y_max_u_balanced_class)]
y_min_u_balanced_class = y_min_u_balanced_class[utils.cols_with_positive_values(y_min_u_balanced_class)]

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_max_u, y_max_u_balanced_class, test_size=0.2, scaling=True)
data_max_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_min_u, y_min_u_balanced_class, test_size=0.2, scaling=True)
data_min_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [18]:
# Regresison data balanced
y_max_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_constr.csv')
exogenous_data_balanced_max_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
y_min_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_constr.csv')
exogenous_data_balanced_min_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_max_u, y_max_u_balanced, test_size=0.2, scaling=True)
data_max_u_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_min_u, y_min_u_balanced, test_size=0.2, scaling=True)
data_min_u_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [19]:
# Classification data sparse
y_max_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_min_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_max_u_bool = y_max_u_bool[utils.cols_with_positive_values(y_max_u_bool)]
y_min_u_bool = y_min_u_bool[utils.cols_with_positive_values(y_min_u_bool)]

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_bool, test_size=0.2, scaling=True)
data_max_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_bool, test_size=0.2, scaling=True)
data_min_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [20]:
# Regression data sparse
y_max_u_sparse = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_constr.csv').drop(columns=['timestamps'])
y_min_u_sparse = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_constr.csv').drop(columns=['timestamps'])

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_sparse, test_size=0.2, scaling=True)
data_max_u_sparse = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_sparse, test_size=0.2, scaling=True)
data_min_u_sparse = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [21]:
max_u_threshold = utils.compute_threshold(y_max_u_sparse)
min_u_threshold = utils.compute_threshold(y_min_u_sparse)

## Prepared Hybrid data sets

### Training


In [22]:
# max u
data_max_u_hybrid = {}
data_max_u_hybrid['X_train'] = pd.concat([data_max_u_balanced['X_train'], data_max_u_bool_balanced['y_train']], axis=1)
data_max_u_hybrid['y_train'] = deepcopy(data_max_u_balanced['y_train'])
# min u 
data_min_u_hybrid = {}
data_min_u_hybrid['X_train'] = pd.concat([data_min_u_sparse['X_train'], data_min_u_bool['y_train']], axis=1)
data_min_u_hybrid['y_train'] = deepcopy(data_min_u_sparse['y_train'])

### Testing

In [23]:
# max u
classifier_max_u_balanced = utils.deserialize_object('pickles\dataset_benchmark\max_u_classifier_balanced')
max_u_class_xgb = deepcopy(classifier_max_u_balanced.strategies[1])
print(classifier_max_u_balanced.strategies[1])
class_result_max_u = data_max_u_bool['y_test'] #max_u_class_xgb.predict(data_max_u_bool)
data_max_u_hybrid['X_test'] = pd.concat([data_max_u_sparse['X_test'], class_result_max_u], axis=1)
data_max_u_hybrid['y_test'] = deepcopy(data_max_u_sparse['y_test'])
# min u 
classifier_min_u_sparse = utils.deserialize_object('pickles\dataset_benchmark\min_u_classifier')
min_u_class_gb = deepcopy(classifier_min_u_sparse.strategies[1])
print(classifier_min_u_sparse.strategies[0])
class_result_min_u = data_min_u_bool['y_test']#min_u_class_gb.predict(data_min_u_bool)
data_min_u_hybrid['X_test'] = pd.concat([data_min_u_sparse['X_test'], class_result_min_u], axis=1)
data_min_u_hybrid['y_test'] = deepcopy(data_min_u_sparse['y_test'])

In [24]:
print('data_max_u_hybrid X_train shape: ', data_max_u_hybrid['X_train'].shape)
print('data_max_u_hybrid y_train shape: ', data_max_u_hybrid['y_train'].shape)
print('data_max_u_hybrid X_test shape: ', data_max_u_hybrid['X_test'].shape)
print('data_max_u_hybrid y_test shape: ', data_max_u_hybrid['y_test'].shape)
print('\n')
print('data_min_u_hybrid X_train shape: ', data_min_u_hybrid['X_train'].shape)
print('data_min_u_hybrid y_train shape: ', data_min_u_hybrid['y_train'].shape)
print('data_min_u_hybrid X_test shape: ', data_min_u_hybrid['X_test'].shape)
print('data_min_u_hybrid y_test shape: ', data_min_u_hybrid['y_test'].shape)


data_max_u_hybrid X_train shape:  (5561, 22)
data_max_u_hybrid y_train shape:  (5561, 34)
data_max_u_hybrid X_test shape:  (9044, 22)
data_max_u_hybrid y_test shape:  (9044, 34)


data_min_u_hybrid X_train shape:  (36172, 21)
data_min_u_hybrid y_train shape:  (36172, 34)
data_min_u_hybrid X_test shape:  (9044, 21)
data_min_u_hybrid y_test shape:  (9044, 34)


In [25]:
data_max_u_hybrid['X_train']

,T,R,wind_speed,wind_direction,season,weekday,last_hour_mean_wind_speed,last_day_mean_wind_direction,last_hour_mean_temperature,cos_hour_day,...,bus_9,bus_10,bus_11,bus_12,bus_13,bus_14,bus_15,bus_16,bus_18,bus_32
0,0.951486,0.245492,0.337958,0.515547,0.000000,0.571429,0.352654,0.506755,0.951845,-0.965926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.979990,0.535954,0.180339,0.904162,0.000000,0.571429,0.176763,0.904904,0.980475,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.939903,0.283968,0.516215,0.013216,0.333333,0.142857,0.523429,0.014206,0.939917,-1.000000,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.937021,0.018382,0.163743,0.192276,1.000000,0.857143,0.164796,0.183076,0.937236,0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.973330,0.419284,0.154116,0.885962,0.666667,0.285714,0.151589,0.889501,0.972570,-1.000000,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,0.957807,0.774332,0.298751,0.926074,0.666667,0.142857,0.306448,0.968075,0.958644,0.707107,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5557,0.948529,0.000000,0.172961,0.076455,0.000000,0.857143,0.190497,0.067510,0.948582,0.707107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5558,0.963519,0.377063,0.189360,0.861642,0.000000,0.428571,0.196625,0.868602,0.964074,-0.965926,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5559,0.962641,0.201991,0.048223,0.033057,0.000000,0.714286,0.047201,0.052769,0.962152,-1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


## Training

In [28]:
balanced_hyper_params = {}
sparse_hyper_params = {}
for file in os.listdir('hyper_params_results_mcc'):
    if file.endswith('.csv') and 'regression_sparse' in file:
        df = pd.read_csv(os.path.join('hyper_params_results', file))
        sparse_hyper_params[file] = df
    elif file.endswith('.csv') and 'regression_balanced' in file:
        df = pd.read_csv(os.path.join('hyper_params_results', file))
        balanced_hyper_params[file] = df
    else:
        pass
import ast
def get_hyper_params_from_df(df):
    output = {}
    for row in df.iterrows():
        if row[1]['params'] != 'value':
            try:
                output[row[1]['params']] = ast.literal_eval(row[1]['value'])
            except :
                output[row[1]['params']] = row[1]['value']
    return output


# max_u
if 'hybrid_regressor_max_u.pickle' not in os.listdir('pickles\hybrid_models_benchmark'):
    print('training max u hybrid model... wait for it...')
    hyper_params = get_hyper_params_from_df(balanced_hyper_params['params_gradient_boost_regression_balanced_max_u.csv'])
    hybrid_regressor_max_u = my_ai.Context(strategy=my_ai.GradientBoostRegressorStrategy(hyper_params)) 
    hybrid_regressor_max_u.fit(data=data_max_u_hybrid)
    utils.serialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u', hybrid_regressor_max_u)
else: 
    print('loading max u hybrid model...')
    hybrid_regressor_max_u = utils.deserialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u')

# min_u 
if 'hybrid_regressor_min_u.pickle' not in os.listdir('pickles\hybrid_models_benchmark'):
    hyper_params = get_hyper_params_from_df(sparse_hyper_params['params_gradient_boost_regression_sparse_min_u.csv'])
    hybrid_regressor_min_u = my_ai.Context(my_ai.GradientBoostRegressorStrategy(hyper_params))
    hybrid_regressor_min_u.fit(data=data_min_u_hybrid)
    utils.serialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_min_u', hybrid_regressor_min_u)
else:
    print('loading min u hybrid model...')
    hybrid_regressor_min_u = utils.deserialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_min_u')

loading max u hybrid model...
loading min u hybrid model...


## Testing

In [29]:
# Load models
# max_u
regression_max_u_focused = utils.deserialize_object('pickles\dataset_benchmark\max_u_regressor_focused')
max_u_reg_lr = deepcopy(regression_max_u_focused.strategies[0])
print(max_u_reg_lr)
reg_result = max_u_reg_lr.predict(data_max_u_sparse)

In [30]:
# max_u
metric = metrics.Metrics()
_threshold = lambda experiment: max_u_threshold / data_max_u_sparse['scaler']['y'] if 'max_u' in experiment else min_u_threshold/ data_min_u_sparse['scaler']['y']
scaled_threshold = _threshold('max_u')
metric.get_prediction_scores(reg_result, data_max_u_sparse['y_test'], threshold=scaled_threshold)
reg_accuracy = metric.hybrid_accuracy
reg_precision = metric.hybrid_precision
reg_recall = metric.hybrid_recall
reg_f1 = metric.hybrid_f1
reg_mcc = metric.hybrid_mcc
reg_evaluation = pd.Series({'accuracy': reg_accuracy, 'precision': reg_precision, 'recall': reg_recall, 'f1': reg_f1, 'mcc': reg_mcc})
# same for hybrid model 
# max_u
hybrid_reg_result = hybrid_regressor_max_u.predict(data_max_u_hybrid)
metric.get_prediction_scores(hybrid_reg_result, data_max_u_hybrid['y_test'], threshold=scaled_threshold)
hybrid_reg_accuracy = metric.hybrid_accuracy
hybrid_reg_precision = metric.hybrid_precision
hybrid_reg_recall = metric.hybrid_recall
hybrid_reg_f1 = metric.hybrid_f1
hybrid_reg_mcc = metric.hybrid_mcc
hybrid_reg_evaluation = pd.Series({'accuracy': hybrid_reg_accuracy, 'precision': hybrid_reg_precision, 'recall': hybrid_reg_recall, 'f1': hybrid_reg_f1, 'mcc': hybrid_reg_mcc})
pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid'])


true_positives_ctr:  4321
true_negatives_ctr:  297379
false_positives_ctr:  5168
false_negatives_ctr:  628
4234062035962222569
true_positives_ctr:  4096
true_negatives_ctr:  297386
false_positives_ctr:  5161
false_negatives_ctr:  853
4133757567112252569


,accuracy,precision,recall,f1,mcc
regression,0.977467,0.366413,0.822844,0.507041,0.540358
hybrid,0.977352,0.349136,0.761696,0.478805,0.506539


In [33]:
def experiment_string(value):
    output = ''
    if 'reg' in value:
        output += 'Regression'
    elif 'hybrid' in value:
        output += 'Hybrid'
    return output
def print_df_in_latex(df):
    for index, row in df.iterrows():
        print( experiment_string(index) + ' & ' + str(round(row['accuracy'], 2)) + ' & ' + str(round(row['precision'], 2)) + ' & ' + str(round(row['recall'], 2)) + ' & ' + str(round(row['f1'], 2)) +' & ' + str(round(row['mcc'], 2)) + ' \\\\')
print_df_in_latex(pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid']))

Regression & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\
Hybrid & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\


In [31]:
# min_u
regression_min_u_sparse = utils.deserialize_object('pickles\dataset_benchmark\min_u_regressor_sparse')
min_u_reg_gb = deepcopy(regression_min_u_sparse.strategies[1])
print(min_u_reg_gb)
reg_result = min_u_reg_gb.predict(data_min_u_sparse)

In [32]:
# min_u
scaled_threshold = _threshold('min_u')
metric.get_prediction_scores(reg_result, data_min_u_sparse['y_test'], threshold=scaled_threshold)
reg_accuracy = metric.hybrid_accuracy
reg_precision = metric.hybrid_precision
reg_recall = metric.hybrid_recall
reg_f1 = metric.hybrid_f1
reg_mcc = metric.hybrid_mcc
reg_evaluation = pd.Series({'recall': reg_recall, 'precision': reg_precision, 'f1': reg_f1, 'accuracy': reg_accuracy, 'mcc': reg_mcc})
# same for hybrid model 
# min_u
hybrid_reg_result = hybrid_regressor_min_u.predict(data_min_u_hybrid)
metric.get_prediction_scores(hybrid_reg_result, data_min_u_hybrid['y_test'], threshold=scaled_threshold)
hybrid_reg_accuracy = metric.hybrid_accuracy
hybrid_reg_precision = metric.hybrid_precision
hybrid_reg_recall = metric.hybrid_recall
hybrid_reg_f1 = metric.hybrid_f1
hybrid_reg_mcc = metric.hybrid_mcc
hybrid_reg_evaluation = pd.Series({'accuracy': hybrid_reg_accuracy, 'precision': hybrid_reg_precision, 'recall': hybrid_reg_recall, 'f1': hybrid_reg_f1, 'mcc': hybrid_reg_mcc})
pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid'])

true_positives_ctr:  5039
true_negatives_ctr:  294582
false_positives_ctr:  6892
false_negatives_ctr:  983
6402070686261354420
true_positives_ctr:  5033
true_negatives_ctr:  294606
false_positives_ctr:  6868
false_negatives_ctr:  989
6386621127097188660


,recall,precision,f1,accuracy,mcc
regression,0.787945,0.353983,0.488506,0.970974,0.516571
hybrid,0.786738,0.354551,0.488814,0.971048,0.516597


In [34]:
print_df_in_latex(pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid']))

Regression & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\
Hybrid & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\
